In [1]:
'''
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

使用管道，特征编码，独热向量，特征转换，特征组合
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

pipeline = Pipeline(stages=[gender_indexer,embark_indexer,gender_encoder,embark_encoder,assembler,log_reg_titanic])
Pipeline.fit
'''

"\nfrom pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)\nfrom pyspark.ml.classification import LogisticRegression\nfrom pyspark.ml import Pipeline\nfrom pyspark.ml.evaluation import BinaryClassificationEvaluator\n\n使用管道，特征编码，独热向量，特征转换，特征组合\ngender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')\ngender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')\n\npipeline = Pipeline(stages=[gender_indexer,embark_indexer,gender_encoder,embark_encoder,assembler,log_reg_titanic])\nPipeline.fit\n"

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

In [3]:
spark = SparkSession.builder.appName("PySpark04").master("local[2]"). \
config("spark.sql.warehouse.dir","file:///E:/input/spark/warehouse").getOrCreate()

In [4]:
data = spark.read.format('csv').load("file:///e:/Download/titanic/train.csv",inferSchema=True,header=True)

In [5]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [7]:
my_cols = data.select(['Survived', 'Pclass', 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'
 ])

In [8]:
final_data = my_cols.na.drop()

In [9]:
# norminal, ordered, descrete, continue, 参考吉米的
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

In [10]:
# StringIndexer
# a b c
# 0 1 2
# one hot encode
# A:[1,0,0], B:[0,1,0], C:[0,0,1]
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex') #性别是norminal,采用StringIndexer
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [11]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [12]:
# 挑选ETL过后的特征组合成特征向量
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],outputCol='features')

In [13]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [14]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [15]:
# 简单管道，像是estimator
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,gender_encoder,embark_encoder,assembler,log_reg_titanic])

In [16]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [17]:
fit_model = pipeline.fit(train_data)

In [18]:
type(assembler)

pyspark.ml.feature.VectorAssembler

In [19]:
results = fit_model.transform(test_data) # transform自动调用prediction

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
#默认参数
my_eval=BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',
    labelCol='label',
    metricName='areaUnderROC')

In [22]:
# 比较预测值和实际值，找到对应标签
my_eval=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [23]:
results.columns # 原始栏位+StringIndexer + Vector + StringIndexer + Vector  + features(VectorAssembler)
# + 新产生的三个栏位'rawPrediction', 'probability','prediction'

['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'SexIndex',
 'EmbarkIndex',
 'SexVec',
 'EmbarkVec',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [24]:
results.show(3)

+--------+------+------+----+-----+-----+------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|  Fare|Embarked|SexIndex|EmbarkIndex|       SexVec|    EmbarkVec|            features|       rawPrediction|         probability|prediction|
+--------+------+------+----+-----+-----+------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|       0|     1|female| 2.0|    1|    2|151.55|       S|     1.0|        0.0|    (1,[],[])|(2,[0],[1.0])|[1.0,0.0,1.0,0.0,...|[-3.5244825637515...|[0.02862359773815...|       1.0|
|       0|     1|  male|18.0|    1|    0| 108.9|       C|     0.0|        1.0|(1,[0],[1.0])|(2,[1],[1.0])|[1.0,1.0,0.0,1.0,...|[-0.8568693890183...|[0.29799383471721...|       1.0|
|       0|     1|  male|19.0|    3|    2| 263.0|       S|     0.0|        0.0|(1,[0],[1.0])|(2,

In [25]:
my_eval.evaluate(results) # AUC

0.8248839009287926

In [26]:
my_eval.

SyntaxError: invalid syntax (<ipython-input-26-21fe2bb826f6>, line 1)

In [ ]:
results.select('SexVec','EmbarkVec',
 'features',
 'rawPrediction',
 'probability',
 'prediction','Survived').show()

In [ ]:
spark.